<a href="https://colab.research.google.com/github/danquio194/AlgorithmsUN2022II/blob/main/Quiz6/Quiz6_ALG_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Group N Quiz 6

## Members

1.   Carlos Isaza
2.   Daniel Quiñones
3.   Juan E. Cepeda


In [1]:
import pandas as pd  
  
# assign data of lists.  
data = {'LMA': [  150, 120, 120 ,90, 80, 40,  103, 95, 95, 105], 
        'SMA': [15,30,12,10,25,30,32,35,25, 35] , 
        'R6M': [ -0.026669057627121146,  -0.0894036394370826 ,  -0.07412753460499291,  -0.17019350214480872, -0.2874441876429889,
                -0.28205321994336763,  -0.17656751726628425,  -0.20114237542777333, -0.21856589322766357, -0.12784107061682198], 
        'R2Y': [-0.6893967581045538, 0.06922550873983395,  0.016829499368699357,  -0.12934369441547033, -0.12469215687567806,
                -0.21040403956146927,  0.026745708712896776,  -0.03614353864292326,  -0.0626398203141331, 0.04499091853956914]}  
  

# Create DataFrame  
df = pd.DataFrame(data)  

df['AR'] = (df['R6M'] + df['R2Y'])/2

df = df.sort_values(by=['AR'], ascending=False)
  
df

,LMA,SMA,R6M,R2Y,AR
1,120,30,-0.089404,0.069226,-0.010089
2,120,12,-0.074128,0.016829,-0.028649
9,105,35,-0.127841,0.044991,-0.041425
6,103,32,-0.176568,0.026746,-0.074911
7,95,35,-0.201142,-0.036144,-0.118643
8,95,25,-0.218566,-0.062640,-0.140603
3,90,10,-0.170194,-0.129344,-0.149769
4,80,25,-0.287444,-0.124692,-0.206068
5,40,30,-0.282053,-0.210404,-0.246229
0,150,15,-0.026669,-0.689397,-0.358033


# Q18 Quick Start

This template shows you the basic steps for taking part to the **Q18 NASDAQ-100 Stock Long-Short contest**.

In [ ]:
import xarray as xr

import qnt.ta as qnta
import qnt.data as qndata
import qnt.output as qnout
import qnt.stats as qns

data = qndata.stocks.load_ndx_data(min_date="2020-10-15")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 200)
sma_fast  = qnta.sma(close, 20)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2020-10-15", None)))
display(stats.to_pandas().tail())

# graph
performance = stats.to_pandas()["equity"]
import qnt.graph as qngraph

qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

100% (18687 of 18687) |##################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (4422536 of 4422536) |##############| Elapsed Time: 0:00:00 Time:  0:00:00


fetched chunk 1/1 2s
Data loaded 2s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,0.713307,0.027522,0.169333,-0.286693,-0.345137,-0.926299,-0.156853,-0.716981,122.0,0.036594,100.319086
2022-10-10,0.721224,0.011099,0.169368,-0.278776,-0.345137,-0.896631,-0.151860,-0.716981,122.0,0.036653,100.410710
2022-10-11,0.729074,0.010884,0.169394,-0.270926,-0.345137,-0.867499,-0.146949,-0.716981,122.0,0.036644,100.410710
2022-10-12,0.729445,0.000509,0.169227,-0.270555,-0.345137,-0.865470,-0.146461,-0.716981,122.0,0.036606,100.410710
2022-10-13,0.719508,-0.013623,0.169310,-0.280492,-0.345137,-0.897992,-0.152039,-0.716981,122.0,0.036579,105.607790


In [ ]:
weights = weights.sel(time=slice("2006-01-01",None))

qnout.check(weights, data, "stocks_nasdaq100")
#qnout.write(weights) # to participate in the competition

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data
The first point(2020-10-15) should be earlier than 2006-01-01
Load data more historical data.
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2020-10-15


Period: 2020-10-15 - 2022-10-13
Sharpe Ratio = -0.8979921493345323


ERROR! The Sharpe Ratio is too low. -0.8979921493345323 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.

Ok. This strategy does not correlate with other strategies.


### 6 months ratio

In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 150)
sma_fast  = qnta.sma(close, 15)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2022-04-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,0.975692,0.025716,0.212423,-0.104842,-0.14313,-0.235186,-0.049959,-0.546218,119.0,0.050439,39.404051
2022-10-10,0.985843,0.010404,0.212085,-0.095529,-0.14313,-0.136841,-0.029022,-0.546218,119.0,0.050235,39.404051
2022-10-11,0.995603,0.009901,0.211695,-0.086574,-0.14313,-0.042452,-0.008987,-0.529412,119.0,0.050053,39.404051
2022-10-12,0.995937,0.000335,0.210840,-0.086268,-0.14313,-0.039080,-0.008240,-0.529412,119.0,0.049912,39.234116
2022-10-13,0.983919,-0.012068,0.210694,-0.097295,-0.14313,-0.152616,-0.032155,-0.529412,119.0,0.049684,52.225999


In [ ]:
weights = weights.sel(time=slice("2022-04-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2022-04-18


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = -0.026669057627121146


ERROR! The Sharpe Ratio is too low. -0.026669057627121146 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.

Ok. This strategy does not correlate with other strategies.


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 120)
sma_fast  = qnta.sma(close, 30)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2022-04-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,0.953649,0.015325,0.192289,-0.105781,-0.132917,-0.489441,-0.094114,-0.243697,119.0,0.046515,47.156283
2022-10-10,0.958137,0.004707,0.191635,-0.101572,-0.132917,-0.441169,-0.084543,-0.260504,119.0,0.046307,47.156283
2022-10-11,0.961846,0.003870,0.190946,-0.098095,-0.132917,-0.401198,-0.076607,-0.294118,119.0,0.046262,47.097284
2022-10-12,0.961511,-0.000348,0.190174,-0.098408,-0.132917,-0.403136,-0.076666,-0.310924,119.0,0.046279,47.231373
2022-10-13,0.952137,-0.009750,0.189888,-0.107199,-0.132917,-0.495799,-0.094147,-0.310924,119.0,0.046223,56.459939


In [ ]:
weights = weights.sel(time=slice("2022-04-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2022-04-18


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = -0.0894036394370826


ERROR! The Sharpe Ratio is too low. -0.0894036394370826 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.



WARNING! This strategy correlates with other strategies and will be rejected.
Modify the strategy to produce the different output.


The number of systems with a larger Sharpe ratio and correlation larger than 0.9: 1
The max correlation value (with systems with a larger Sharpe ratio): 0.9515343408477616
Current sharpe ratio(3y): -0.497570887304474

Correlated examples:

Name               Coefficient    Sharpe ratio
---------------  -------------  --------------
Q18 Quick Start       0.951534       -0.255745


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 120)
sma_fast  = qnta.sma(close, 12)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2022-04-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,0.955582,0.021207,0.202096,-0.114025,-0.151044,-0.446745,-0.090285,-0.462185,119.0,0.055625,34.474812
2022-10-10,0.962356,0.007089,0.201544,-0.107744,-0.151044,-0.378069,-0.076197,-0.445378,119.0,0.055492,34.343984
2022-10-11,0.968537,0.006423,0.200947,-0.102013,-0.151044,-0.315494,-0.063397,-0.428571,119.0,0.055384,34.219361
2022-10-12,0.968040,-0.000514,0.200135,-0.102474,-0.151044,-0.319188,-0.063881,-0.428571,119.0,0.055198,34.095957
2022-10-13,0.958119,-0.010248,0.199841,-0.111673,-0.151044,-0.413513,-0.082637,-0.445378,119.0,0.054938,45.264722


In [ ]:
weights = weights.sel(time=slice("2022-04-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2022-04-18


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = -0.07412753460499291


ERROR! The Sharpe Ratio is too low. -0.07412753460499291 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.



WARNING! This strategy correlates with other strategies and will be rejected.
Modify the strategy to produce the different output.


The number of systems with a larger Sharpe ratio and correlation larger than 0.9: 1
The max correlation value (with systems with a larger Sharpe ratio): 0.9535125314589379
Current sharpe ratio(3y): -0.4150106304302533

Correlated examples:

Name               Coefficient    Sharpe ratio
---------------  -------------  --------------
Q18 Quick Start       0.953513       -0.255745


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 90)
sma_fast  = qnta.sma(close, 10)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2022-04-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,0.906699,0.024274,0.194505,-0.155457,-0.181472,-0.948690,-0.184525,-0.478992,119.0,0.065758,31.846704
2022-10-10,0.913474,0.007473,0.194062,-0.149146,-0.181472,-0.878594,-0.170502,-0.445378,119.0,0.065544,31.746721
2022-10-11,0.919592,0.006697,0.193557,-0.143448,-0.181472,-0.815276,-0.157802,-0.478992,119.0,0.065486,31.598804
2022-10-12,0.920143,0.000600,0.192782,-0.142934,-0.181472,-0.807162,-0.155607,-0.495798,119.0,0.065346,31.385728
2022-10-13,0.909402,-0.011673,0.192647,-0.152939,-0.181472,-0.904465,-0.174243,-0.462185,119.0,0.065128,36.454252


In [ ]:
weights = weights.sel(time=slice("2022-04-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2022-04-18


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = -0.17019350214480872


ERROR! The Sharpe Ratio is too low. -0.17019350214480872 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.

Ok. This strategy does not correlate with other strategies.


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 80)
sma_fast  = qnta.sma(close, 25)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2022-04-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,0.865166,0.017404,0.177038,-0.168256,-0.18637,-1.470836,-0.260394,-0.378151,119.0,0.058497,39.183672
2022-10-10,0.870399,0.006049,0.176610,-0.163225,-0.18637,-1.411407,-0.249269,-0.378151,119.0,0.058869,39.576500
2022-10-11,0.874174,0.004336,0.176060,-0.159596,-0.18637,-1.367812,-0.240817,-0.378151,119.0,0.058589,39.576500
2022-10-12,0.874821,0.000740,0.175367,-0.158974,-0.18637,-1.357058,-0.237983,-0.445378,119.0,0.058243,39.576500
2022-10-13,0.863450,-0.012998,0.175484,-0.169906,-0.18637,-1.459984,-0.256203,-0.478992,119.0,0.058470,41.416863


In [ ]:
weights = weights.sel(time=slice("2022-04-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2022-04-18


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = -0.2874441876429889


ERROR! The Sharpe Ratio is too low. -0.2874441876429889 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.

Ok. This strategy does not correlate with other strategies.


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 40)
sma_fast  = qnta.sma(close, 30)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2022-04-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,0.850513,0.028342,0.184950,-0.150253,-0.17938,-1.547682,-0.286244,-0.731092,119.0,0.088051,23.280104
2022-10-10,0.858886,0.009845,0.184876,-0.141887,-0.17938,-1.458472,-0.269637,-0.697479,119.0,0.087802,23.258723
2022-10-11,0.867943,0.010545,0.184880,-0.132839,-0.17938,-1.362299,-0.251862,-0.680672,119.0,0.087586,23.199683
2022-10-12,0.869089,0.001320,0.184165,-0.131694,-0.17938,-1.347142,-0.248096,-0.630252,119.0,0.087143,23.161474
2022-10-13,0.859865,-0.010613,0.183923,-0.140909,-0.17938,-1.426772,-0.262416,-0.613445,119.0,0.087018,25.717566


In [ ]:
weights = weights.sel(time=slice("2022-04-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2022-04-18


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = -0.28205321994336763


ERROR! The Sharpe Ratio is too low. -0.28205321994336763 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.

Ok. This strategy does not correlate with other strategies.


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 102)
sma_fast  = qnta.sma(close, 32)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2022-04-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,0.916040,0.011274,0.180756,-0.130771,-0.14383,-0.923506,-0.166930,-0.159664,119.0,0.052260,42.111912
2022-10-10,0.919739,0.004038,0.180140,-0.127262,-0.14383,-0.881030,-0.158708,-0.159664,119.0,0.052540,42.179271
2022-10-11,0.922435,0.002932,0.179477,-0.124703,-0.14383,-0.849454,-0.152458,-0.193277,119.0,0.052304,42.179271
2022-10-12,0.922518,0.000090,0.178755,-0.124624,-0.14383,-0.845689,-0.151171,-0.260504,119.0,0.052276,42.156130
2022-10-13,0.912701,-0.010642,0.178605,-0.133940,-0.14383,-0.941708,-0.168194,-0.277311,119.0,0.052564,47.623625


In [ ]:
weights = weights.sel(time=slice("2022-04-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2022-04-18


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = -0.17656751726628425


ERROR! The Sharpe Ratio is too low. -0.17656751726628425 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.

Ok. This strategy does not correlate with other strategies.


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 95)
sma_fast  = qnta.sma(close, 35)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2022-04-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,0.904584,0.008999,0.177933,-0.137153,-0.153261,-1.059288,-0.188482,-0.092437,119.0,0.052455,43.604005
2022-10-10,0.907996,0.003772,0.177321,-0.133899,-0.153261,-1.019312,-0.180745,-0.126050,119.0,0.052597,43.631546
2022-10-11,0.910632,0.002904,0.176675,-0.131383,-0.153261,-0.987843,-0.174527,-0.176471,119.0,0.052614,43.567130
2022-10-12,0.911521,0.000976,0.175977,-0.130536,-0.153261,-0.975177,-0.171609,-0.226891,119.0,0.052724,43.460075
2022-10-13,0.902590,-0.009798,0.175745,-0.139054,-0.153261,-1.062140,-0.186665,-0.260504,119.0,0.052881,46.857461


In [ ]:
weights = weights.sel(time=slice("2022-04-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2022-04-18


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = -0.20114237542777333


ERROR! The Sharpe Ratio is too low. -0.20114237542777333 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.

Ok. This strategy does not correlate with other strategies.


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 95)
sma_fast  = qnta.sma(close, 25)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2022-04-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,0.892347,0.016222,0.181741,-0.144444,-0.162858,-1.161965,-0.211176,-0.327731,119.0,0.054281,41.650636
2022-10-10,0.897623,0.005912,0.181255,-0.139386,-0.162858,-1.103194,-0.199959,-0.327731,119.0,0.054283,41.644108
2022-10-11,0.901038,0.003804,0.180637,-0.136112,-0.162858,-1.064275,-0.192247,-0.344538,119.0,0.054037,41.644108
2022-10-12,0.901400,0.000402,0.179915,-0.135765,-0.162858,-1.057137,-0.190195,-0.361345,119.0,0.053859,41.725742
2022-10-13,0.892307,-0.010087,0.179677,-0.144482,-0.162858,-1.142262,-0.205239,-0.411765,119.0,0.053741,45.832873


In [ ]:
weights = weights.sel(time=slice("2022-04-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2022-04-18


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = -0.21856589322766357


ERROR! The Sharpe Ratio is too low. -0.21856589322766357 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.

Ok. This strategy does not correlate with other strategies.


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 105)
sma_fast  = qnta.sma(close, 35)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2022-04-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,0.936849,0.008573,0.182636,-0.115923,-0.133242,-0.695557,-0.127034,-0.092437,119.0,0.049025,46.546866
2022-10-10,0.940405,0.003796,0.181988,-0.112567,-0.133242,-0.654958,-0.119195,-0.126050,119.0,0.049195,46.319727
2022-10-11,0.942233,0.001944,0.181279,-0.110842,-0.133242,-0.633116,-0.114770,-0.159664,119.0,0.049254,46.125992
2022-10-12,0.942935,0.000745,0.180554,-0.110180,-0.133242,-0.623403,-0.112558,-0.159664,119.0,0.049249,46.281745
2022-10-13,0.935572,-0.007808,0.180134,-0.117128,-0.133242,-0.697464,-0.125637,-0.176471,119.0,0.049050,51.481892


In [ ]:
weights = weights.sel(time=slice("2022-04-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2022-04-18


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = -0.12784107061682198


ERROR! The Sharpe Ratio is too low. -0.12784107061682198 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.



WARNING! This strategy correlates with other strategies and will be rejected.
Modify the strategy to produce the different output.


The number of systems with a larger Sharpe ratio and correlation larger than 0.9: 1
The max correlation value (with systems with a larger Sharpe ratio): 0.9015462160251976
Current sharpe ratio(3y): -0.6998612093365508

Correlated examples:

Name               Coefficient    Sharpe ratio
---------------  -------------  --------------
Q18 Quick Start       0.901546       -0.255745


### 2 years ratios

In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 150)
sma_fast  = qnta.sma(close, 15)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2020-10-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/1 0s
Data loaded 0s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,0.773773,0.027682,0.169248,-0.226227,-0.261701,-0.717796,-0.121486,-0.716981,122.0,0.041893,80.006414
2022-10-10,0.782444,0.011205,0.169281,-0.217556,-0.261701,-0.687078,-0.116309,-0.716981,122.0,0.041868,80.006414
2022-10-11,0.790788,0.010664,0.169295,-0.209212,-0.261701,-0.657801,-0.111362,-0.698113,122.0,0.041848,80.006414
2022-10-12,0.791074,0.000362,0.169127,-0.208926,-0.261701,-0.656263,-0.110992,-0.698113,122.0,0.041837,79.880972
2022-10-13,0.780702,-0.013111,0.169197,-0.219298,-0.261701,-0.689397,-0.116644,-0.698113,122.0,0.041801,81.936986


In [ ]:
weights = weights.sel(time=slice("2020-10-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data
The first point(2020-10-15) should be earlier than 2006-01-01
Load data more historical data.
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2020-10-15


Period: 2020-10-15 - 2022-10-13
Sharpe Ratio = -0.6893967581045538


ERROR! The Sharpe Ratio is too low. -0.6893967581045538 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.

Ok. This strategy does not correlate with other strategies.


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 120)
sma_fast  = qnta.sma(close, 30)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2020-10-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,1.055837,0.014779,0.132338,-0.102449,-0.128731,0.210211,0.027819,-0.243697,135.0,0.035525,82.099113
2022-10-10,1.060632,0.004542,0.132242,-0.098373,-0.128731,0.227707,0.030113,-0.260504,135.0,0.035496,82.099113
2022-10-11,1.064594,0.003735,0.132135,-0.095005,-0.128731,0.242062,0.031985,-0.294118,135.0,0.035507,82.017802
2022-10-12,1.064236,-0.000336,0.132004,-0.095309,-0.128731,0.240496,0.031746,-0.310924,135.0,0.035532,81.921880
2022-10-13,1.054222,-0.009410,0.132046,-0.103822,-0.128731,0.203012,0.026807,-0.310924,135.0,0.035540,91.573007


In [ ]:
weights = weights.sel(time=slice("2020-10-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2020-10-15


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = 0.06922550873983395


ERROR! The Sharpe Ratio is too low. 0.06922550873983395 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.



WARNING! This strategy correlates with other strategies and will be rejected.
Modify the strategy to produce the different output.


The number of systems with a larger Sharpe ratio and correlation larger than 0.9: 1
The max correlation value (with systems with a larger Sharpe ratio): 0.9429567643886066
Current sharpe ratio(3y): 0.20301232646455564

Correlated examples:

Name               Coefficient    Sharpe ratio
---------------  -------------  --------------
Q18 Quick Start       0.942957        0.198821


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 120)
sma_fast  = qnta.sma(close, 12)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2020-10-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,1.010617,0.020427,0.134975,-0.110368,-0.1462,0.039620,0.005348,-0.462185,135.0,0.043241,58.290089
2022-10-10,1.017523,0.006834,0.134926,-0.104288,-0.1462,0.065175,0.008794,-0.445378,135.0,0.043233,58.223814
2022-10-11,1.023824,0.006193,0.134861,-0.098742,-0.1462,0.088339,0.011913,-0.428571,135.0,0.043231,58.159029
2022-10-12,1.023317,-0.000495,0.134727,-0.099188,-0.1462,0.086381,0.011638,-0.428571,135.0,0.043209,58.094414
2022-10-13,1.013203,-0.009884,0.134777,-0.108091,-0.1462,0.048918,0.006593,-0.445378,135.0,0.043169,67.763981


In [ ]:
weights = weights.sel(time=slice("2020-10-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2020-10-15


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = 0.016829499368699357


ERROR! The Sharpe Ratio is too low. 0.016829499368699357 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.



WARNING! This strategy correlates with other strategies and will be rejected.
Modify the strategy to produce the different output.


The number of systems with a larger Sharpe ratio and correlation larger than 0.9: 1
The max correlation value (with systems with a larger Sharpe ratio): 0.9174880595513154
Current sharpe ratio(3y): 0.04891799785627629

Correlated examples:

Name               Coefficient    Sharpe ratio
---------------  -------------  --------------
Q18 Quick Start       0.917488        0.198821


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 90)
sma_fast  = qnta.sma(close, 10)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2020-10-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,0.903155,0.023317,0.132002,-0.159820,-0.184702,-0.379844,-0.050140,-0.478992,135.0,0.052700,45.673631
2022-10-10,0.909644,0.007185,0.131973,-0.153783,-0.184702,-0.353169,-0.046609,-0.445378,135.0,0.052673,45.633648
2022-10-11,0.915502,0.006440,0.131924,-0.148333,-0.184702,-0.329235,-0.043434,-0.478992,135.0,0.052684,45.567317
2022-10-12,0.916030,0.000577,0.131794,-0.147842,-0.184702,-0.326817,-0.043073,-0.495798,135.0,0.052675,45.483079
2022-10-13,0.905744,-0.011229,0.131896,-0.157411,-0.184702,-0.366863,-0.048388,-0.462185,135.0,0.052647,51.163472


In [ ]:
weights = weights.sel(time=slice("2020-10-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2020-10-15


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = -0.12934369441547033


ERROR! The Sharpe Ratio is too low. -0.12934369441547033 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.

Ok. This strategy does not correlate with other strategies.


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 80)
sma_fast  = qnta.sma(close, 25)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2020-10-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,0.915196,0.016710,0.125222,-0.173241,-0.19054,-0.349507,-0.043766,-0.378151,135.0,0.044703,59.202221
2022-10-10,0.920514,0.005812,0.125168,-0.168436,-0.19054,-0.326627,-0.040883,-0.378151,135.0,0.044821,59.237929
2022-10-11,0.924350,0.004167,0.125081,-0.164971,-0.19054,-0.310161,-0.038795,-0.378151,135.0,0.044780,59.237929
2022-10-12,0.925008,0.000712,0.124957,-0.164376,-0.19054,-0.307112,-0.038376,-0.445378,135.0,0.044722,59.237929
2022-10-13,0.913451,-0.012494,0.125139,-0.174816,-0.19054,-0.354322,-0.044339,-0.478992,135.0,0.044806,64.049328


In [ ]:
weights = weights.sel(time=slice("2020-10-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2020-10-15


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = -0.12469215687567806


ERROR! The Sharpe Ratio is too low. -0.12469215687567806 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.

Ok. This strategy does not correlate with other strategies.


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 40)
sma_fast  = qnta.sma(close, 30)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2020-10-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,0.850807,0.027203,0.123357,-0.220711,-0.246377,-0.635185,-0.078354,-0.731092,135.0,0.076780,28.213193
2022-10-10,0.858856,0.009460,0.123427,-0.213339,-0.246377,-0.598078,-0.073819,-0.697479,135.0,0.076741,28.202215
2022-10-11,0.867561,0.010136,0.123525,-0.205365,-0.246377,-0.558323,-0.068967,-0.680672,135.0,0.076710,28.181665
2022-10-12,0.868662,0.001269,0.123406,-0.204356,-0.246377,-0.552977,-0.068241,-0.630252,135.0,0.076622,28.169150
2022-10-13,0.859797,-0.010206,0.123484,-0.212477,-0.246377,-0.590254,-0.072887,-0.613445,135.0,0.076613,31.519671


In [ ]:
weights = weights.sel(time=slice("2020-10-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2020-10-15


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = -0.21040403956146927


ERROR! The Sharpe Ratio is too low. -0.21040403956146927 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.

Ok. This strategy does not correlate with other strategies.


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 102)
sma_fast  = qnta.sma(close, 32)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2020-10-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,1.023506,0.010859,0.127746,-0.126611,-0.139255,0.092390,0.011802,-0.159664,135.0,0.038357,73.658025
2022-10-10,1.027488,0.003891,0.127647,-0.123213,-0.139255,0.107803,0.013761,-0.159664,135.0,0.038453,73.474722
2022-10-11,1.030391,0.002825,0.127534,-0.120736,-0.139255,0.118969,0.015173,-0.193277,135.0,0.038423,73.474722
2022-10-12,1.030480,0.000087,0.127407,-0.120660,-0.139255,0.119196,0.015186,-0.260504,135.0,0.038443,73.367071
2022-10-13,1.019910,-0.010257,0.127490,-0.129680,-0.139255,0.077856,0.009926,-0.277311,135.0,0.038543,80.467094


In [ ]:
weights = weights.sel(time=slice("2020-10-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2020-10-15


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = 0.026745708712896776


ERROR! The Sharpe Ratio is too low. 0.026745708712896776 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.

Ok. This strategy does not correlate with other strategies.


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 95)
sma_fast  = qnta.sma(close, 35)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2020-10-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,0.976126,0.008661,0.125906,-0.133123,-0.148703,-0.096332,-0.012129,-0.092437,135.0,0.039330,71.234857
2022-10-10,0.979670,0.003631,0.125806,-0.129976,-0.148703,-0.081859,-0.010298,-0.126050,135.0,0.039390,71.115270
2022-10-11,0.982410,0.002796,0.125696,-0.127543,-0.148703,-0.070701,-0.008887,-0.176471,135.0,0.039421,71.012121
2022-10-12,0.983333,0.000940,0.125573,-0.126723,-0.148703,-0.066908,-0.008402,-0.226891,135.0,0.039474,70.854760
2022-10-13,0.974055,-0.009435,0.125625,-0.134962,-0.148703,-0.104147,-0.013083,-0.260504,135.0,0.039540,77.049233


In [ ]:
weights = weights.sel(time=slice("2020-10-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2020-10-15


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = -0.03614353864292326


ERROR! The Sharpe Ratio is too low. -0.03614353864292326 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.

Ok. This strategy does not correlate with other strategies.


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 95)
sma_fast  = qnta.sma(close, 25)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2020-10-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,0.954831,0.015591,0.127729,-0.149828,-0.167425,-0.180632,-0.023072,-0.327731,135.0,0.041432,65.229302
2022-10-10,0.960259,0.005685,0.127666,-0.144994,-0.167425,-0.158466,-0.020231,-0.327731,135.0,0.041458,65.161038
2022-10-11,0.963773,0.003659,0.127566,-0.141865,-0.167425,-0.144153,-0.018389,-0.344538,135.0,0.041423,65.161038
2022-10-12,0.964146,0.000386,0.127439,-0.141534,-0.167425,-0.142517,-0.018162,-0.361345,135.0,0.041404,65.150964
2022-10-13,0.954789,-0.009705,0.127496,-0.149865,-0.167425,-0.179707,-0.022912,-0.411765,135.0,0.041400,71.840968


In [ ]:
weights = weights.sel(time=slice("2020-10-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2020-10-15


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = -0.0626398203141331


ERROR! The Sharpe Ratio is too low. -0.0626398203141331 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.

Ok. This strategy does not correlate with other strategies.


In [ ]:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

close     = data.sel(field="close")
is_liquid = data.sel(field="is_liquid")
sma_slow  = qnta.sma(close, 105)
sma_fast  = qnta.sma(close, 35)
weights   = xr.where(sma_slow < sma_fast, 1, -1)
weights   = weights * is_liquid

weights = qnout.clean(weights, data, "stocks_nasdaq100")

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice("2020-10-15", None)))
display(stats.to_pandas().tail())

fetched chunk 1/5 0s
fetched chunk 2/5 0s
fetched chunk 3/5 0s
fetched chunk 4/5 0s
fetched chunk 5/5 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-10-07,1.035031,0.008263,0.127719,-0.112229,-0.128996,0.137333,0.017540,-0.092437,135.0,0.037100,77.695567
2022-10-10,1.038819,0.003660,0.127616,-0.108980,-0.128996,0.151860,0.019380,-0.126050,135.0,0.037166,77.471858
2022-10-11,1.040766,0.001874,0.127495,-0.107310,-0.128996,0.159232,0.020301,-0.159664,135.0,0.037204,77.310856
2022-10-12,1.041513,0.000718,0.127369,-0.106669,-0.128996,0.161957,0.020628,-0.159664,135.0,0.037227,77.248308
2022-10-13,1.033671,-0.007530,0.127357,-0.113396,-0.128996,0.131360,0.016730,-0.176471,135.0,0.037202,84.626301


In [ ]:
weights = weights.sel(time=slice("2020-10-15",None))

qnout.check(weights, data, "stocks_nasdaq100")

Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the data for the slippage calculation.
Add 15 extra data points to the data head (load data more historical data).
WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2020-10-15


Period: 2006-01-03 - 2022-10-13
Sharpe Ratio = 0.04499091853956914


ERROR! The Sharpe Ratio is too low. 0.04499091853956914 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


Check correlation.



WARNING! This strategy correlates with other strategies and will be rejected.
Modify the strategy to produce the different output.


The number of systems with a larger Sharpe ratio and correlation larger than 0.9: 1
The max correlation value (with systems with a larger Sharpe ratio): 0.9051798478251598
Current sharpe ratio(3y): 0.13135951254672862

Correlated examples:

Name               Coefficient    Sharpe ratio
---------------  -------------  --------------
Q18 Quick Start        0.90518        0.198821


# Description

## 1. Data

![data_example](./img/data.jpg)

The variable **qndata.stocks.load_ndx_data(tail=period)** is an xarray.DataArray structure which contains historical market data for the last (tail=period) days and whose coordinates are:

* **time**: a date in format yyyy-mm-dd;
* **field**: an attribute, for example the opening daily price;
* **asset**: the identifying symbol for the asset, for example NAS:APPL for Apple.

More details on xarray can be found at https://xarray.pydata.org/en/stable/.

## 2. Strategy. Weights allocation


![weights_example](./img/weights.jpg)

For each date, the algorithm calculates the portfolio weights which should be used at the opening of the next day's trading.

Quantiacs uses an exposure-based backtester. The trading algorithm should define the fractions of capital which will be distributed to the assets (allocation weights). A **positive** weight means a long position (**buy**), a **negative** value means a short position (**sell**).

> Note that algorithm decisions can use all data available at the close of the session, and will be applied at the opening of the next day's session. The chosen allocation weights are translated to positions (number of contracts to be bought/sold) immediately after the close of the session and transactions are executed at the open of the next day.


## 3. Performance estimation

![statistic_example](./img/statistic.png)

After we have built the algorithm, we can evaluate its performance calculating statistics.

We can display the values of statistical indicators on a cumulative basis, assuming that we have 1M USD at the starting point.

The call will produce:

* **equity**: the cumulative value of profits and losses since inception (1M USD);
* **relative_return**: the relative daily variation of **equity**;
* **volatility**: the volatility of the investment since inception (i.e. the annualized standard deviation of the daily returns);
* **underwater**: the time evolution of drawdowns;
* **max_drawdown**: the absolute minimum of the underwater chart;
* **sharpe_ratio**: the annualized Sharpe ratio since inception; the value must be larger than 1 for taking part to contests;
* **mean_return**: the annualized mean return of the investment since inception;
* **bias**: the daily asymmetry between long and short exposure: 1 for a long-only system, -1 for a short-only one;
* **instruments**: the number of instruments which get allocations on a given day;
* **avg_turnover**: the average turnover;
* **avg_holding_time**: the average holding time in days.

A detailed explanation can be found inspecting the source code for the library in your directory at **/qnt/stats.py**

Moreover we can produce a chart which shows the cumulative profits and losses.

# Alternative way of writing strategies

A quick prototpye can be written using a single-pass implementation where all the time series are processed at once. Beware that unintentional forward looking can take place!

> If strategy sharpe coefficient in the competition differs from the sharpe coefficient in jupyter, rewrite the strategy for Multi-Pass

<br>


<table style="text-align: left ; font-size: 1em ; min-width: 1000px">
<tbody><tr>
<td style="text-align: left"> Single-pass (quick check of the result)</td> <td style="text-align: left"> Multi-Pass equivalent</td>
</tr>
<tr>
<td style="text-align: left ; min-width: 500px">

<pre><code class="cm-s-ipython language-python"><span style=" color: #008000;  font-weight: bold;">import</span> <span class="cm-variable">xarray</span> <span style=" color: #008000;  font-weight: bold;">as</span> <span class="cm-variable">xr</span>

<span style=" color: #008000;  font-weight: bold;">import</span> <span class="cm-variable">qnt</span>.<span class="cm-property">ta</span> <span style=" color: #008000;  font-weight: bold;">as</span> <span class="cm-variable">qnta</span>
<span style=" color: #008000;  font-weight: bold;">import</span> <span class="cm-variable">qnt</span>.<span class="cm-property">data</span> <span style=" color: #008000;  font-weight: bold;">as</span> <span class="cm-variable">qndata</span>
<span style=" color: #008000;  font-weight: bold;">import</span> <span class="cm-variable">qnt</span>.<span class="cm-property">output</span> <span style=" color: #008000;  font-weight: bold;">as</span> <span class="cm-variable">qnout</span>
<span style=" color: #008000;  font-weight: bold;">import</span> <span class="cm-variable">qnt</span>.<span class="cm-property">stats</span> <span style=" color: #008000;  font-weight: bold;">as</span> <span class="cm-variable">qns</span>

<span class="cm-variable">data</span> <span class="cm-operator">=</span> <span class="cm-variable">qndata</span>.<span class="cm-property">stocks</span>.<span class="cm-property">load_ndx_data</span>(<span class="cm-variable">min_date</span><span class="cm-operator">=</span><span style=" color: #BA2121;">"2006-01-01"</span>)

<span class="cm-variable">close</span>     <span class="cm-operator">=</span> <span class="cm-variable">data</span>.<span class="cm-property">sel</span>(<span class="cm-variable">field</span><span class="cm-operator">=</span><span style=" color: #BA2121;">"close"</span>)
<span class="cm-variable">is_liquid</span> <span class="cm-operator">=</span> <span class="cm-variable">data</span>.<span class="cm-property">sel</span>(<span class="cm-variable">field</span><span class="cm-operator">=</span><span style=" color: #BA2121;">"is_liquid"</span>)
<span class="cm-variable">sma_slow</span>  <span class="cm-operator">=</span> <span class="cm-variable">qnta</span>.<span class="cm-property">sma</span>(<span class="cm-variable">close</span>, <span style=" color: #080;">200</span>)
<span class="cm-variable">sma_fast</span>  <span class="cm-operator">=</span> <span class="cm-variable">qnta</span>.<span class="cm-property">sma</span>(<span class="cm-variable">close</span>, <span style=" color: #080;">20</span>)
<span class="cm-variable">weights</span>   <span class="cm-operator">=</span> <span class="cm-variable">xr</span>.<span class="cm-property">where</span>(<span class="cm-variable">sma_slow</span> <span class="cm-operator">&lt;</span> <span class="cm-variable">sma_fast</span>, <span style=" color: #080;">1</span>, <span class="cm-operator">-</span><span style=" color: #080;">1</span>)
<span class="cm-variable">weights</span>   <span class="cm-operator">=</span> <span class="cm-variable">weights</span> <span class="cm-operator">*</span> <span class="cm-variable">is_liquid</span>

<span class="cm-variable">weights</span> <span class="cm-operator">=</span> <span class="cm-variable">qnout</span>.<span class="cm-property">clean</span>(<span class="cm-variable">weights</span>, <span class="cm-variable">data</span>, <span style=" color: #BA2121;">"stocks_nasdaq100"</span>)

<span class="cm-variable">qnout</span>.<span class="cm-property">check</span>(<span class="cm-variable">weights</span>, <span class="cm-variable">data</span>, <span style=" color: #BA2121;">"stocks_nasdaq100"</span>)

<span class="cm-variable">qnout</span>.<span class="cm-property">write</span>(<span class="cm-variable">weights</span>)

<span class="cm-comment"># calc stats</span>
<span class="cm-variable">stats</span> <span class="cm-operator">=</span> <span class="cm-variable">qns</span>.<span class="cm-property">calc_stat</span>(<span class="cm-variable">data</span>, <span class="cm-variable">weights</span>.<span class="cm-property">sel</span>(<span class="cm-variable">time</span><span class="cm-operator">=</span><span class="cm-builtin">slice</span>(<span style=" color: #BA2121;">"2006-01-01"</span>,<span style=" color: #008000;  font-weight: bold;">None</span>)))
<span class="cm-variable">stats</span>.<span class="cm-property">to_pandas</span>().<span class="cm-property">tail</span>()</code></pre>
</td>
<td style="text-align: left ; min-width: 500px">

<pre><code class="cm-s-ipython language-python"><span style=" color: #008000;  font-weight: bold;">import</span> <span class="cm-variable">xarray</span> <span style=" color: #008000;  font-weight: bold;">as</span> <span class="cm-variable">xr</span>

<span style=" color: #008000;  font-weight: bold;">import</span> <span class="cm-variable">qnt</span>.<span class="cm-property">ta</span> <span style=" color: #008000;  font-weight: bold;">as</span> <span class="cm-variable">qnta</span>
<span style=" color: #008000;  font-weight: bold;">import</span> <span class="cm-variable">qnt</span>.<span class="cm-property">backtester</span> <span style=" color: #008000;  font-weight: bold;">as</span> <span class="cm-variable">qnbt</span>
<span style=" color: #008000;  font-weight: bold;">import</span> <span class="cm-variable">qnt</span>.<span class="cm-property">data</span> <span style=" color: #008000;  font-weight: bold;">as</span> <span class="cm-variable">qndata</span>



<span style=" color: #008000;  font-weight: bold;">def</span> <span style=" color:  #00f;">load_data</span>(<span class="cm-variable">period</span>):
    <span style=" color: #008000;  font-weight: bold;">return</span> <span class="cm-variable">qndata</span>.<span class="cm-property">stocks</span>.<span class="cm-property">load_ndx_data</span>(<span class="cm-variable">tail</span><span class="cm-operator">=</span><span class="cm-variable">period</span>)



<span style=" color: #008000;  font-weight: bold;">def</span> <span style=" color:  #00f;">strategy</span>(<span class="cm-variable">data</span>):
    <span class="cm-variable">close</span>     <span class="cm-operator">=</span> <span class="cm-variable">data</span>.<span class="cm-property">sel</span>(<span class="cm-variable">field</span><span class="cm-operator">=</span><span style=" color: #BA2121;">"close"</span>)
    <span class="cm-variable">is_liquid</span> <span class="cm-operator">=</span> <span class="cm-variable">data</span>.<span class="cm-property">sel</span>(<span class="cm-variable">field</span><span class="cm-operator">=</span><span style=" color: #BA2121;">"is_liquid"</span>)
    <span class="cm-variable">sma_slow</span>  <span class="cm-operator">=</span> <span class="cm-variable">qnta</span>.<span class="cm-property">sma</span>(<span class="cm-variable">close</span>, <span style=" color: #080;">200</span>).<span class="cm-property">isel</span>(<span class="cm-variable">time</span><span class="cm-operator">=</span><span class="cm-operator">-</span><span style=" color: #080;">1</span>)
    <span class="cm-variable">sma_fast</span>  <span class="cm-operator">=</span> <span class="cm-variable">qnta</span>.<span class="cm-property">sma</span>(<span class="cm-variable">close</span>, <span style=" color: #080;">20</span>).<span class="cm-property">isel</span>(<span class="cm-variable">time</span><span class="cm-operator">=</span><span class="cm-operator">-</span><span style=" color: #080;">1</span>)
    <span class="cm-variable">weights</span>   <span class="cm-operator">=</span> <span class="cm-variable">xr</span>.<span class="cm-property">where</span>(<span class="cm-variable">sma_slow</span> <span class="cm-operator">&lt;</span> <span class="cm-variable">sma_fast</span>, <span style=" color: #080;">1</span>, <span class="cm-operator">-</span><span style=" color: #080;">1</span>)
    <span class="cm-variable">weights</span>   <span class="cm-operator">=</span> <span class="cm-variable">weights</span> <span class="cm-operator">*</span> <span class="cm-variable">is_liquid</span>
    <span style=" color: #008000;  font-weight: bold;">return</span> <span class="cm-variable">weights</span>



<span class="cm-variable">weights</span> <span class="cm-operator">=</span> <span class="cm-variable">qnbt</span>.<span class="cm-property">backtest</span>(
    <span class="cm-variable">competition_type</span> <span class="cm-operator">=</span> <span style=" color: #BA2121;">"stocks_nasdaq100"</span>,
    <span class="cm-variable">load_data</span>        <span class="cm-operator">=</span> <span class="cm-variable">load_data</span>,
    <span class="cm-variable">lookback_period</span>  <span class="cm-operator">=</span> <span style=" color: #080;">365</span><span class="cm-operator">*</span><span style=" color: #080;">4</span>,
    <span class="cm-variable">start_date</span>       <span class="cm-operator">=</span> <span style=" color: #BA2121;">"2006-01-01"</span>,
    <span class="cm-variable">strategy</span>         <span class="cm-operator">=</span> <span class="cm-variable">strategy</span>,
    <span class="cm-variable">analyze</span>          <span class="cm-operator">=</span> <span style=" color: #008000;  font-weight: bold;">True</span>,
    <span class="cm-variable">build_plots</span>      <span class="cm-operator">=</span> <span style=" color: #008000;  font-weight: bold;">True</span>
)</code></pre>
</td>
</tr>
</tbody></table>

# Example of a strategy using technical analysis indicators

The example of a strategy with a sharpe ratio of 0.9615 trading 215 financial instruments

The strategy use sma, ema, adl (Advance–Decline line)




```python
from IPython.display import display
import xarray as xr
import qnt.data as qndata
import qnt.output as qnout
import qnt.ta as qnta
import qnt.stats as qns

data = qndata.stocks.load_ndx_data(min_date="2005-01-01")


def get_strategy_1(data, params):
    buy = 1
    not_trade = 0
    close = data.sel(field="close")

    strategy_1 = xr.where(qnta.sma(close, params[1]) > qnta.sma(close, params[0]), buy, not_trade)
    strategy_2 = xr.where(qnta.ema(close, params[2]) > qnta.ema(close, params[3]), buy, not_trade)

    weights = strategy_1 * strategy_2 * data.sel(field="is_liquid")
    weights = weights / 100.0
    return weights.fillna(0)


def get_strategy_2(data, params):
    buy = 1
    not_trade = 0
    close = data.sel(field="close") * data.sel(field="is_liquid")

    adl = qnta.ad_line(close) * 1.0
    adl_dif = adl.shift(time=params[0]) - adl.shift(time=params[1])
    positive_trend = adl_dif > 0
    strategy_1 = xr.where(positive_trend, buy, not_trade)

    weights = strategy_1 * data.sel(field="is_liquid")
    return weights.fillna(0)


weights_1 = get_strategy_1(data, [25, 40, 12, 132])  # 0.6108887689714039 Sharpe Ratio
weights_2 = get_strategy_2(data, [34, 183])  # 0.6012686822757577

weights_all = 2 * weights_1 * weights_2 - weights_1
weights = qnout.clean(output=weights_all, data=data, kind="stocks_nasdaq100")  # 0.9615

qnout.check(weights, data, "stocks_nasdaq100")
qnout.write(weights)
```

# What libraries are available?

```python
# Import basic libraries.
import xarray as xr
import pandas as pd
import numpy as np

# Import quantnet libraries.
import qnt.data    as qndata  # load and manipulate data
import qnt.output as output   # manage output
import qnt.backtester as qnbt # backtester
import qnt.stats   as qnstats # statistical functions for analysis
import qnt.graph   as qngraph # graphical tools
import qnt.ta      as qnta    # indicators library
```

# May I import libraries?

Yes, please refer to the file **init.ipynb** in your home directory. You can for example use:

```
! conda install -y scikit-learn
```

# How to load data?

Daily stock data for the **Q18 Nasdaq-100** contest can be loaded using:
```python
data = qndata.stocks.load_ndx_data(tail = 17*365, dims = ("time", "field", "asset"))
```

Cryptocurrency daily data used for the Q16/Q17 contests can be loaded using:
```python
data = qndata.cryptodaily.load_data(tail = 17*365, dims = ("time", "field", "asset"))
```

Futures data for the Q15 contest can be loaded using:
```python
data= qndata.futures.load_data(tail = 17*365, dims = ("time", "field", "asset"))
```

BTC Futures data for the Q15 contest can be loaded using:
```python
data= qndata.cryptofutures.load_data(tail = 17*365, dims = ("time", "field", "asset"))
```

# How to view a list of all tickers?

```python
data.asset.to_pandas().to_list()
```

# How to see which fields are available?

```python
data.field.to_pandas().to_list()
```

# How to load specific tickers?

```python
data = qndata.stocks.load_ndx_data(tail=17 * 365, assets=["NAS:AAPL", "NAS:AMZN"])
```

# How to select specific tickers after loading all data?

```python
def get_data_filter(data, assets):
    filler= data.sel(asset=assets)
    return filler

get_data_filter(data, ["NAS:AAPL", "NAS:AMZN"])
```

# How to get the prices for the previous day?

```python
qnta.shift(data.sel(field="open"), periods=1)
```

or:

```python
data.sel(field="open").shift(time=1)
```

# How to get the Sharpe ratio?

```python
import qnt.stats as qnstats

def get_sharpe(market_data, weights):
    rr = qnstats.calc_relative_return(market_data, weights)
    sharpe = qnstats.calc_sharpe_ratio_annualized(rr).values[-1]
    return sharpe

sharpe = get_sharpe(data, weights) # weights.sel(time=slice("2006-01-01",None))
```

# How do I get a list of the top 3 assets ranked by Sharpe ratio?

```python
import qnt.stats as qnstats

data = qndata.stocks.load_ndx_data(tail = 17*365, dims = ("time", "field", "asset"))

def get_best_instruments(data, weights, top_size):
    # compute statistics:
    stats_per_asset = qnstats.calc_stat(data, weights, per_asset=True)
    # calculate ranks of assets by "sharpe_ratio":
    ranks = (-stats_per_asset.sel(field="sharpe_ratio")).rank("asset")
    # select top assets by rank "top_period" days ago:
    top_period = 1
    rank = ranks.isel(time=-top_period)
    top = rank.where(rank <= top_size).dropna("asset").asset

    # select top stats:
    top_stats = stats_per_asset.sel(asset=top.values)

    # print results:
    print("SR tail of the top assets:")
    display(top_stats.sel(field="sharpe_ratio").to_pandas().tail())

    print("avg SR = ", top_stats[-top_period:].sel(field="sharpe_ratio").mean("asset")[-1].item())
    display(top_stats)
    return top_stats.coords["asset"].values

get_best_instruments(data, weights, 3)
```

# How can I check the results for only the top 3 assets ranked by Sharpe ratio?

Select the top assets and then load their data:

```python
best_assets= get_best_instruments(data, weights, 3)

data= qndata.stocks.load_ndx_data(tail = 17*365, assets=best_assets)
```

# How can prices be processed?

Simply import standard libraries, for example **numpy**:

```python
import numpy as np

high= np.log(data.sel(field="high"))
```

# How can you reduce slippage impace when trading?

Just apply some technique to reduce turnover:

```python
def get_lower_slippage(weights, rolling_time=6):
    return weights.rolling({"time": rolling_time}).max()

improved_weights = get_lower_slippage(weights, rolling_time=6)
```

# How to use technical analysis indicators?

For available indicators see the source code of the library: /qnt/ta

## ATR

```python
def get_atr(data, days=14):
    high = data.sel(field="high") * 1.0 
    low  = data.sel(field="low") * 1.0 
    close= data.sel(field="close") * 1.0

    return qnta.atr(high, low, close, days)

atr= get_atr(data, days=14)
```

## EMA

```python
prices= data.sel(field="high")
prices_ema= qnta.ema(prices, 15)
```

## TRIX

```python
prices= data.sel(field="high")
prices_trix= qnta.trix(prices, 15)
```

## ADL and EMA

```python
adl= qnta.ad_line(data.sel(field="close")) * 1.0 
adl_ema= qnta.ema(adl, 18)
```

# How can you check the quality of your strategy?

```python
import qnt.output as qnout
qnout.check(weights, data, "stocks_nasdaq100")
```

or

```python
stat= qnstats.calc_stat(data, weights)
display(stat.to_pandas().tail())
```

or

```python
import qnt.graph   as qngraph
statistics= qnstats.calc_stat(data, weights)
display(statistics.to_pandas().tail())

performance= statistics.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

display(statistics[-1:].sel(field = ["sharpe_ratio"]).transpose().to_pandas())
qnstats.print_correlation(weights, data)

```

# An example using pandas

One can work with pandas DataFrames at intermediate steps and at the end convert them to xarray data structures:

```python
def get_price_pct_change(prices):
    prices_pandas = prices.to_pandas()
    assets = data.coords["asset"].values
    for asset in assets:
        prices_pandas[asset] = prices_pandas[asset].pct_change()
    return prices_pandas

prices = data.sel(field="close") * 1.0
prices_pct_change = get_price_pct_change(prices).unstack().to_xarray()
```

# Disable widget scrolling
```javascript
%%javascript
window.IPython && (IPython.OutputArea.prototype._should_scroll = function(lines) { return false; })
// disable widget scrolling
```

# How can I combine datasets?

Let us suppose that we want to use some Futures data as external indicators for taking positions on Nasdaq-100 stocks. This can easily be achieved as follows:

```python
import xarray as xr
import numpy as np

import qnt.backtester as qnbt
import qnt.data as qndata
import qnt.ta as qnta



def load_data(period):
    futures = qndata.futures.load_data(tail=period, assets=["F_DX"]).isel(asset=0)
    stocks  = qndata.stocks.load_ndx_data(tail=period)
    return {"futures": futures, "stocks": stocks}, futures.time.values



def window(data, max_date: np.datetime64, lookback_period: int):
    min_date = max_date - np.timedelta64(lookback_period, "D")
    return {
        "futures": data["futures"].sel(time=slice(min_date, max_date)),
        "stocks":  data["stocks"].sel(time=slice(min_date, max_date)),
    }



def strategy(data):
    close_futures = data["futures"].sel(field="close")
    close_stocks  = data["stocks"].sel(field="close")
    sma20 = qnta.sma(close_futures, 20).isel(time=-1)
    sma20_stocks = qnta.sma(close_stocks, 20).isel(time=-1)
    is_liquid = data["stocks"].sel(field="is_liquid").isel(time=-1)
    weights = xr.where(sma20 < sma20_stocks, 1, -1)
    weights = weights * is_liquid 
    weights = weights / 100.0
    return weights



qnbt.backtest(
    competition_type= "stocks_nasdaq100",
    load_data= load_data,
    lookback_period= 365,
    start_date= "2006-01-01",
    strategy= strategy,
    window= window
)

```

# How to submit a strategy to the competition?

Check that weights are fine:

```python
import qnt.output as qnout
qnout.check(weights, data, "stocks_nasdaq100")
```

If everything is ok, write the weights to file:

```python
qnout.write(weights)
```

In your **personal account**:

* **choose** a strategy;
* click on the **Submit** button;
* select the type of competition.

At the beginning you will find the strategy under the **Checking** area:

* **Sent strategies** > **Checking**. 

If technical checks are successful, the strategy will go under the **Candidates** area:

* **Sent strategies** > **Candidates**. 

Otherwise it will be **Filtered**:

* **Sent strategies** > **Filtered**

and you should inspect error and warning messages. 

Note that a strategy under the **Candidates** area should have a Sharpe ratio larger than 1 for being eligible for a prize. Please check warning messages in your **Candidates** area!


Please note that:

* Your trading algorithm can open short and long positions.

* At each point in time your algorithm can trade all or a subset of the stocks which at that point of time are or were part of the NASDAQ-100 stock index. Note that the composition of this set changes in time, and Quantiacs provides you with an appropriate filter function for selecting them.

* The Sharpe ratio of your system since January 1st, 2006, has to be larger than 1.

* Your system cannot be a copy of the current examples. We run a correlation filter on the submissions and detect duplicates.

More details on the rules can be found [here](https://quantiacs.com/contest).


# How to find good parameters for my algorithm?

See examples

* Trading System Optimization
* Trading System Optimization by Asset


Read more on our [article](https://quantiacs.com/community/topic/29/optimizing-and-monitoring-a-trading-system-with-quantiacs).




# The main reasons for submission rejection

[Detailed explanation with examples](https://quantiacs.com/documentation/en/user_guide/passFilters.html).

## Missed call to write_output

Save algorithm weights,  run code

```python
qnt.output.write(weights)
```

## Not eligible send to contest. In-Sample Sharpe must be larger than 1

Improve your algorithm. Аor example, you can use sections and get an algorithm that will pass the filter

* Example of a strategy using technical analysis indicators
* How do I get a list of the top 3 assets ranked by Sharpe ratio?

**Need help?** Check the [**Documentation**](https://quantiacs.com/documentation/en/) and find solutions/report problems in the [**Forum**](https://quantiacs.com/community/categories) section.


## Not enough bid information.

Run code

```python
min_time = weights.time[abs(weights).fillna(0).sum('asset')> 0].min()
min_time
```
> min_time must be less than or equal to January 1, 2006.

If min_time is larger than the starting date, we recommend to fill the starting values of the time series with non-vanishing values, for example a simple buy-and-hold strategy.

```python
min_time = weights.time[abs(weights).fillna(0).sum('asset')> 0].min()
weights_new = xr.where(weights.time < min_time, 1, weights)
qnt.output.write(weights_new)
```
